Based on Chainer's MNIST example.

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L


# Network definition
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

    
n_unit = 1000
model = L.Classifier(MLP(n_unit, 10))    


# Setup an optimizer
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)


# Load the MNIST dataset
train, test = chainer.datasets.get_mnist()
train_iter = chainer.iterators.SerialIterator(train, 100)
test_iter = chainer.iterators.SerialIterator(test, 100, repeat=False, shuffle=False)

In [ ]:
from chainer.training.extensions import Evaluator, LogReport

from chaineripy.extensions import PrintReport
from chaineripy.extensions import ProgressBar


# Set up a trainer
updater = chainer.training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = chainer.training.Trainer(updater, (20, 'epoch'))


# Evaluate the model with the test dataset for each epoch
trainer.extend(Evaluator(test_iter, model, device=-1))
# Write a log of evaluation statistics for each epoch
trainer.extend(LogReport())


# print the statistics to a IPython widget
trainer.extend(PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
# show the progress bar on a IPython widget
trainer.extend(ProgressBar(update_interval=10))


trainer.run()